In [1]:
script_path = getwd()                                # %exclude_jupyterlab%
script_path                                          # %exclude_jupyterlab%

[1] "/Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna"

In [23]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_seurat_opt.R --genome="hg38" --data.dir=/Volumes/ccrsf-static/singlecell_projects/DwightNissley_CS035576_6scRNASeq_101223/Analysis_CPMV/Analysis/1_B3C2_NC/outs/filtered_feature_bc_matrix --outdir="test_dir"'),  # %exclude_jupyterlab%
       intern = T) # %exclude_jupyterlab%

character(0)

In [3]:
library(Seurat)
library(Matrix)
library(MASS)
library(dplyr)
library(reshape2)
library(ggplot2)
library(URD)
library(cluster)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'gplots':
  method         from 
  reorder.factor gdata



In [4]:
opt = readRDS("./test_dir/opt.rds")                                 # %exclude_jupyterlab%

In [5]:
opt                                                              

$min.cells
[1] 0

$min.features
[1] 0

$data.dir
[1] "/Volumes/ccrsf-static/singlecell_projects/DwightNissley_CS035576_6scRNASeq_101223/Analysis_CPMV/Analysis/1_B3C2_NC/outs/filtered_feature_bc_matrix"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$outdir
[1] "test_dir"

$help
[1] FALSE

In [6]:
system(paste0("mkdir -p ", opt$outdir))
setwd(opt$outdir) 

In [7]:
count_mtx = Read10X(data.dir = opt$data.dir)

In [8]:
count_mtx <- count_mtx[(nrow(count_mtx)/2):nrow(count_mtx), 1:1000]                # %exclude_jupyterlab%

In [9]:
seur <- CreateSeuratObject(counts = count_mtx,
                                 min.cells = opt$min.cells,
                                 min.features = opt$min.features,
                                 project = opt$sampleid)

In [10]:
rm(count_mtx)

In [11]:
#find mitochondrial genes
if (opt$genome == "mm10") {
        seur[["percent.mito"]] <- PercentageFeatureSet(seur, pattern="^mt-")
} else if (opt$genome == "hg19" | opt$genome == "hg38") {
        seur[["percent.mito"]] <- PercentageFeatureSet(seur, pattern="^MT-")
}


In [12]:
head(seur@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mito
,<fct>,<dbl>,<int>,<dbl>
AAACCTGAGAAACCAT-1,scRNA,4350,1716,4.275862
AAACCTGAGCGTCAAG-1,scRNA,1174,655,4.173765
AAACCTGAGCTGATAA-1,scRNA,2847,1003,1.615736
AAACCTGAGGACAGAA-1,scRNA,1558,877,5.584082
AAACCTGAGGCGACAT-1,scRNA,3070,1332,4.267101
AAACCTGAGTACGCCC-1,scRNA,2729,1079,4.323928


In [13]:
png("VlnPlot_PreFilter.png", height=7, width=7, units='in', res=200)
VlnPlot(seur, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3)
dev.off()

Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


png 
  2

In [14]:
plot1 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "percent.mito")
plot2 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
png("FeatureScatter_PreFilter.png", height=7, width=10, units='in', res=200)
plot1 | plot2
dev.off()

png 
  2

In [15]:
cS <- data.frame(libSize=seur$nCount_RNA, geneDetect=seur$nFeature_RNA)
p_hi <- 1e-3 #p-value for filtering doublets
p_lo <- 1e-2 #p-value for filtering poor libraries
fitLibSize <- fitdistr(cS$libSize,"negative binomial")
umi.upper.limit <- qnbinom(p_hi,size=fitLibSize$estimate["size"],
                 mu=fitLibSize$estimate["mu"],lower.tail=F)
umi.lower.limit <- qnbinom(p_lo,size=fitLibSize$estimate["size"],
                 mu=fitLibSize$estimate["mu"],lower.tail=T)
fitGeneDetect <- fitdistr(cS$geneDetect,"negative binomial")
gene.upper.limit <- qnbinom(p_hi,size=fitGeneDetect$estimate["size"],
                 mu=fitGeneDetect$estimate["mu"],lower.tail=F)
gene.lower.limit <- qnbinom(p_lo,size=fitGeneDetect$estimate["size"],
                 mu=fitGeneDetect$estimate["mu"],lower.tail=T)

cur.mad <- mad(seur$percent.mito)
cur.med <- median(seur$percent.mito)
diff.val <- 4 * cur.mad
mito.upper.limit <- cur.med + diff.val

temp_doublets <- (cS$libSize > umi.upper.limit) | (cS$geneDetect > gene.upper.limit) #doublets IDed based on high library size or genes detected
temp_crapLibs <- (cS$libSize < umi.lower.limit) | (cS$geneDetect < gene.lower.limit) #poor libraries IDed based on low library size or genes detected

Warning message in densfun(x, parm[1], parm[2], ...):
“NaNs produced”


In [16]:
print(mito.upper.limit)


[1] 11.96038


In [17]:
filter_summary <- t(data.frame(c("Doublets"=sum(temp_doublets), "Poor-Quality"=sum(temp_crapLibs), "Mitochondrial"=sum(seur$percent.mito > mito.upper.limit), "Total Filtered"=sum(temp_doublets | temp_crapLibs | seur$percent.mito > mito.upper.limit))))

In [18]:
rownames(filter_summary) = opt$sampleid
filter_summary

,Doublets,Poor-Quality,Mitochondrial,Total Filtered
scRNA,11,29,11,50


In [19]:
write.table(filter_summary, 'FilterNumbers.csv', sep=',', quote=FALSE, row.names=FALSE)

In [20]:
seur <- subset(seur, subset = nFeature_RNA > gene.lower.limit & nFeature_RNA < gene.upper.limit)
seur <- subset(seur, subset = nCount_RNA > umi.lower.limit & nCount_RNA < umi.upper.limit)
seur <- subset(seur, subset = percent.mito < mito.upper.limit)

In [21]:
write.table(data.frame(umi.upper.limit, umi.lower.limit, gene.upper.limit, gene.lower.limit, mito.upper.limit), 
            'FilterThresholds.csv', sep=',', quote=FALSE, row.names = FALSE)

In [22]:
png("VlnPlot_Filtered.png", height=7, width=7, units='in', res=200)
VlnPlot(seur, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3)
dev.off()

Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


png 
  2

In [23]:
#new plot
plot1 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "percent.mito")
plot2 <- FeatureScatter(seur, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
png("FeatureScatter_PostFilter.png", height=7, width=10, units='in', res=300)
plot1 | plot2
dev.off()

png 
  2

In [24]:
seur <- SCTransform(seur, vars.to.regress = "percent.mito", return.only.var.genes = FALSE, verbose = FALSE)

In [25]:
saveRDS(seur, file = "seur_10x_preprocessed_object.rds")

In [26]:
###URD
if (dim(seur)[[2]] > 10000) {
  numPCs <- 50
} else if (dim(seur)[[2]] < 500) {
  numPCs <- 10
} else {
  #inputTags <- as.matrix(read.csv(expressionFile, row.names = 1))
  mat1 <- as.matrix(GetAssayData(seur))
  cat("createURD...")
  test <- suppressWarnings(createURD(count.data = mat1, min.cells=3, min.counts=3)) # )
  cat("calcPCA...")
  test <- suppressWarnings(calcPCA(test, mp.factor = 2))
  write.table(test@pca.sig,"URD.txt")
  png("URD.png", height=7, width=7, units='in', res=300)
  pcSDPlot(test)
  dev.off()

  numPCs <- max(10, sum(test@pca.sig))
}

createURD...

2024-04-24 15:42:44.678892: Filtering cells by number of genes.

2024-04-24 15:42:45.397299: Filtering genes by number of cells.

2024-04-24 15:42:46.210632: Filtering genes by number of counts across entire data.

2024-04-24 15:42:46.970242: Filtering genes by maximum observed expression.

2024-04-24 15:42:47.762715: Creating URD object.

2024-04-24 15:42:48.37798: Determining normalization factors.

2024-04-24 15:42:49.088711: Normalizing and log-transforming the data.

'as(<dgeMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").

2024-04-24 15:42:50.858241: Finishing setup of the URD object.

2024-04-24 15:42:50.997969: All done.



calcPCA...[1] "2024-04-24 15:42:51.000045: Centering and scaling data."
[1] "2024-04-24 15:42:51.955693: Removing genes with no variation."
[1] "2024-04-24 15:42:52.254821: Calculating PCA."
[1] "2024-04-24 15:42:53.752586: Estimating significant PCs."
[1] "Marchenko-Pastur eigenvalue null upper bound: 14.0277971069299"
[1] "5 PCs have eigenvalues larger than 2 times null upper bound."
[1] "Storing 10 PCs."


In [27]:
top10 <- head(VariableFeatures(seur), 10)
# plot variable features with and without labels
plot1 <- VariableFeaturePlot(seur)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
png('VariableFeatures.png', height=7, width=10, units='in', res=300)
plot1 | plot2
dev.off()

When using repel, set xnudge and ynudge to 0 for optimal results



png 
  2

In [28]:
seur <- RunPCA(seur, 
               features = VariableFeatures(seur), 
               npcs=max(20,numPCs), do.print = TRUE, 
               ndims.print = 1:5, nfeatures.print = 5)

PC_ 1 
Positive:  IFI30, FOS, FTL, LYZ, TYROBP 
Negative:  CCL5, NKG7, RPL13, RPS3, RPL10 
PC_ 2 
Positive:  RPL13, RPS5, RPL39, CCR7, RPL10 
Negative:  NKG7, CCL5, CST7, GZMB, CCL4 
PC_ 3 
Positive:  CD79A, MS4A1, IGHM, CD37, LINC00926 
Negative:  CD3E, IL32, LDHB, CD3D, NELL2 
PC_ 4 
Positive:  PDE3B, RUNX1, NEAT1, SORL1, MALAT1 
Negative:  CCL5, B2M, NKG7, TYROBP, TMSB4X 
PC_ 5 
Positive:  TYROBP, CD7, KLRF1, CCR7, KLRB1 
Negative:  CCL5, IL32, GZMH, KLRG1, CD3E 



In [29]:
pdf("VizPCAPlot.pdf")
for (i in  seq(1,20,2)) {
  j = i + 1
  print(VizDimLoadings(seur, i:j))
}
dev.off()

png 
  2

In [30]:
#plot PCA
pdf("AllPCAPlot.pdf")
for (i in c(1:10)) {
  print(DimPlot(seur, dims=c(i, i+1), reduction="pca"))
}
dev.off()

png 
  2

In [31]:
#PC heatmap
pdf("PC_HeatmapPlot.pdf")
for (i in c(1:10)) {
  DimHeatmap(seur, dims = i, cells = 500, balanced = TRUE)
}
dev.off()

#make PC elbow plot
pdf("PC_ElbowPlot.pdf")
ElbowPlot(seur)
dev.off()

png 
  2

png 
  2

In [32]:
resolutions <- c(0.1, 0.3, 0.6, 0.8)

seur <- FindNeighbors(seur, dims=1:numPCs)
seur <- RunTSNE(seur, dims=1:numPCs)
write.csv(Embeddings(seur, reduction='tsne'), file = "tSNECoordinates.csv")
seur <- RunUMAP(seur, dims=1:numPCs)
write.csv(Embeddings(seur, reduction='umap'), file = "UMAPCoordinates.csv")

Computing nearest neighbor graph

Computing SNN

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
15:43:09 UMAP embedding parameters a = 0.9922 b = 1.112

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’

15:43:09 Read 949 rows and found 10 numeric columns

15:43:09 Using Annoy for neighbor search, n_neighbors = 30

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’

15:43:09 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


In [33]:
runRes <- c()
tsnePlots <- list()
umapPlots <- list()
for (res in resolutions) {
  seur <- FindClusters(seur, dims=1:numPCs, resolution = res, print.output = 0, save.SNN = T)
  tsne <- TSNEPlot(seur) + ggtitle(paste(numPCs,"PCs_res", res, sep="")) +
    theme(plot.title = element_text(hjust = 0.5))
  tsnePlots[[as.character(res)]] <- tsne
  png(paste("TSNEPlotwith",numPCs,"PCs_", res, ".png", sep=""), height=7, width=7, units='in', res=300)
  print(tsne)
  dev.off()
  umap <- DimPlot(seur, reduction="umap") + ggtitle(paste(numPCs,"PCs_res", res, sep="")) +
    theme(plot.title = element_text(hjust = 0.5))
  umapPlots[[as.character(res)]] <- umap
  png(paste("UMAPPlotwith",numPCs,"PCs_", res, ".png", sep=""), height=7, width=7, units='in', res=300)
  print(umap)
  dev.off()

  try({
    seur.markers <- FindAllMarkers(object = seur, logfc.threshold = 0.25, only.pos=TRUE)
    write.csv(seur.markers %>% group_by(cluster) %>% top_n(-100,
                                                           p_val), paste("top100markers_pc", numPCs, "_res", res, ".csv", sep = ""))
    saveRDS(seur.markers, paste("markers_res", res, ".rds", sep = ""))
    runRes <- append(runRes, res)})
}



Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output


Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output




Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 949
Number of edges: 30498

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9586
Number of communities: 4
Elapsed time: 0 seconds


Calculating cluster 0

For a (much!) faster implementation of the Wilcoxon Rank Sum Test,
(default method for FindMarkers) please install the presto package
--------------------------------------------
install.packages('devtools')
devtools::install_github('immunogenomics/presto')
--------------------------------------------
After installation of presto, Seurat will automatically use the more 
efficient implementation (no further action necessary).
This message will be shown once per session

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output


Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output




Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 949
Number of edges: 30498

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9164
Number of communities: 6
Elapsed time: 0 seconds


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output


Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output




Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 949
Number of edges: 30498

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8648
Number of communities: 6
Elapsed time: 0 seconds


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output


Warning message:
“The following arguments are not used: dims, print.output, save.SNN”
Suggested parameter: verbose instead of print.output




Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 949
Number of edges: 30498

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8303
Number of communities: 6
Elapsed time: 0 seconds


Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5



In [34]:
#save object
saveRDS(seur, file = "seur_10x_cluster_object.rds")

In [35]:
pdf("TSNEPlots.pdf")
for (res in tsnePlots){
  print(res)
}
dev.off()

png 
  2

In [36]:
pdf("UMAPPlots.pdf")
for (res in umapPlots){
  print(res)
}
dev.off()

png 
  2

In [37]:
##Create Silhoutte Plots
for (res in runRes){
  coord <- Embeddings(seur, reduction.type='pca')[,1:numPCs]
  Idents(seur) <- seur@meta.data[[paste0('SCT_snn_res.', res)]]
  clusters <- Idents(seur)
  d <- dist(coord, method="euclidean")
  sil<-silhouette(as.numeric(clusters), dist=d)
  #silPlot <- recordPlot()
  pdf(paste0("SilhouettePlot_res",res,".pdf"))#, height=7, width=7, units='in', res=300)
  plot(sil, col=as.factor(clusters[order(clusters, decreasing=FALSE)]), main=paste("Silhouette plot of Seurat clustering - resolution ", res, sep=""), lty=2)
  abline(v=mean(sil[,3]), col="red4", lty=2)
  dev.off()
}

##Remove resolutions that failed marker generation
print(runRes)
for (res in setdiff(resolutions, runRes)){
  seur@meta.data[paste0('SCT_snn_res.', res)] <- NULL
}

write(min(runRes), "minRes.txt")

Warning message:
“The following arguments are not used: reduction.type”
Warning message:
“The following arguments are not used: reduction.type”
Warning message:
“The following arguments are not used: reduction.type”
Warning message:
“The following arguments are not used: reduction.type”


[1] 0.1 0.3 0.6 0.8


In [38]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] cluster_2.1.6      URD_1.1.1          ggplot2_3.5.0      reshape2_1.4.4    
 [5] dplyr_1.1.4        MASS_7.3-60.0.1    Matrix_1.6-5       Seurat_5.0.1      
 [9] Seu

In [2]:
notebook_prefix = "sc_seurat"                                                         # %exclude_jupyterlab%
notebook_name = paste0(notebook_prefix, ".ipynb")                                     # %exclude_jupyterlab%
notebook_r = paste0(script_path, "/", paste0(notebook_prefix, ".r"))                  # %exclude_jupyterlab%
notebook_path = paste0(script_path, "/", notebook_name)                               # %exclude_jupyterlab%
opt_name = paste0(script_path, "/", sub(".ipynb", "_opt.R", notebook_name))           # %exclude_jupyterlab%
output = paste0(script_path, "/", sub(".ipynb", ".prod.R", notebook_name))            # %exclude_jupyterlab%
cmd1 = paste0("jupyter nbconvert --to script --output ",                              # %exclude_jupyterlab%
             notebook_prefix, ' ', notebook_path, "> /dev/null 2>&1 ")                # %exclude_jupyterlab%
cmd1                                                                                  # %exclude_jupyterlab%
system(cmd1, intern = TRUE)                                                            # %exclude_jupyterlab%

[1] "jupyter nbconvert --to script --output sc_seurat /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_seurat.ipynb> /dev/null 2>&1 "

character(0)

In [3]:
cmd2 = paste0('cat -s ', opt_name, ' ', notebook_r,                                      # %exclude_jupyterlab%
             ' |grep -v exclude_jupyterlab > ', output,  ' 2>&1')                     # %exclude_jupyterlab%
cmd2                                                                                  # %exclude_jupyterlab%
system(cmd2, intern = T)                                                              # %exclude_jupyterlab%
system(paste0("rm ", notebook_r))                                                     # %exclude_jupyterlab%  

[1] "cat -s /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_seurat_opt.R /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_seurat.r |grep -v exclude_jupyterlab > /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_seurat.prod.R 2>&1"

character(0)